In [ ]:
from wiki import get_all_links
from dotenv import load_dotenv
from neo4j import GraphDatabase
import os

load_dotenv()

True

In [50]:
URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
DATABASE = "wikirace"
driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

In [46]:
def get_unexplored():
    records = driver.execute_query("""
            MATCH (a: Article {explored: FALSE}) 
            RETURN a.name AS name
            """,
        database_ = DATABASE,
    ).records
    
    return [record.data()["name"] for record in records]

In [47]:
def create_article(parent, child):
    driver.execute_query("""
        MATCH (p:Article {name: $parent})
        MERGE (c:Article {name: $child})
        MERGE (p)-[:parent_of]->(c)
        """,
        parent=parent,
        child=child,
        database_=DATABASE,
    )

In [53]:
def update_explored(name):
    driver.execute_query("""
        MATCH (a:Article {name: $name})
        SET a.explored = TRUE
        """,
        name=name,
        database_=DATABASE,
    )

In [48]:
driver.close()